In [15]:
# Setup and imports
import sys
import time
import os
from typing import Optional, Literal
from dataclasses import dataclass, field
from IPython.display import HTML, display, Markdown
import re

sys.path.insert(0, '..')
from dotenv import load_dotenv
load_dotenv('../.env')

# Import clients
from openai import OpenAI
import anthropic

from src.minimax_client import MiniMaxClient


In [16]:
# Fetch available models dynamically from each API
def fetch_available_models() -> dict:
    """Fetch available models from each provider's API."""
    available = {
        "minimax": [],
        "openai": [],
        "anthropic": []
    }
    
    # OpenAI - has a models.list() endpoint
    openai_key = os.getenv("OPENAI_API_KEY")
    if openai_key:
        try:
            openai_client = OpenAI(api_key=openai_key)
            models = openai_client.models.list()
            # Filter for chat models (gpt-*, o1-*)
            chat_models = [
                m.id for m in models.data 
                if m.id.startswith(('gpt-4', 'gpt-3.5', 'o1', 'o3'))
                and not m.id.endswith(('-instruct', '-vision-preview'))
                and 'audio' not in m.id
                and 'realtime' not in m.id
            ]
            available["openai"] = sorted(set(chat_models), reverse=True)
            print(f"✓ OpenAI: Found {len(available['openai'])} chat models")
        except Exception as e:
            print(f"✗ OpenAI: Could not fetch models - {e}")
    else:
        print("⚠️ OpenAI: No API key set")
    
    # Anthropic - no public models list API, use known models
    # The API doesn't expose a models list endpoint, so we use known models
    anthropic_key = os.getenv("ANTHROPIC_API_KEY")
    if anthropic_key:
        # These are the currently available Anthropic models
        # You can verify by trying to use them
        available["anthropic"] = [
            "claude-sonnet-4-20250514",
            "claude-3-5-sonnet-20241022", 
            "claude-3-5-haiku-20241022",
            "claude-3-opus-20240229",
            "claude-3-haiku-20240307",
        ]
        print(f"✓ Anthropic: {len(available['anthropic'])} known models (no list API)")
    else:
        print("⚠️ Anthropic: No API key set")
    
    # MiniMax - check if models endpoint exists
    minimax_key = os.getenv("MINIMAX_API_KEY")
    if minimax_key:
        try:
            # MiniMax uses OpenAI-compatible API, try to list models
            minimax_client = OpenAI(
                api_key=minimax_key,
                base_url="https://api.minimax.io/v1"
            )
            models = minimax_client.models.list()
            available["minimax"] = [m.id for m in models.data]
            print(f"✓ MiniMax: Found {len(available['minimax'])} models")
        except Exception as e:
            # Fallback to known models
            available["minimax"] = ["MiniMax-M2.1"]
            print(f"⚠️ MiniMax: Using known models (list API not available)")
    else:
        print("⚠️ MiniMax: No API key set")
    
    return available

print("🔍 Fetching available models from APIs...\n")
AVAILABLE_MODELS = fetch_available_models()

print("\n📋 Available Models:")
for provider, models in AVAILABLE_MODELS.items():
    if models:
        print(f"\n  {provider.upper()}:")
        for model in models[:15]:  # Show first 15 to avoid clutter
            print(f"    • {model}")
        if len(models) > 15:
            print(f"    ... and {len(models) - 15} more")


🔍 Fetching available models from APIs...

✓ OpenAI: Found 48 chat models
✓ Anthropic: 5 known models (no list API)
⚠️ MiniMax: Using known models (list API not available)

📋 Available Models:

  MINIMAX:
    • MiniMax-M2.1

  OPENAI:
    • o3-pro-2025-06-10
    • o3-pro
    • o3-mini-2025-01-31
    • o3-mini
    • o3-deep-research-2025-06-26
    • o3-deep-research
    • o3-2025-04-16
    • o3
    • o1-pro-2025-03-19
    • o1-pro
    • o1-2024-12-17
    • o1
    • gpt-4o-transcribe-diarize
    • gpt-4o-transcribe
    • gpt-4o-search-preview-2025-03-11
    ... and 33 more

  ANTHROPIC:
    • claude-sonnet-4-20250514
    • claude-3-5-sonnet-20241022
    • claude-3-5-haiku-20241022
    • claude-3-opus-20240229
    • claude-3-haiku-20240307


In [17]:
@dataclass
class CompletionResult:
    """Stores results from a model completion."""
    model_name: str
    provider: str
    content: str
    completion_time: float  # seconds
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int
    error: Optional[str] = None
    
    @property
    def tokens_per_second(self) -> float:
        return self.completion_tokens / self.completion_time if self.completion_time > 0 else 0
    
    @property
    def success(self) -> bool:
        return self.error is None
    
    def summary(self) -> str:
        if self.error:
            return f"""
╔══════════════════════════════════════════════════════════════╗
║  {self.provider}: {self.model_name}
╠══════════════════════════════════════════════════════════════╣
║  ❌ Error: {self.error[:50]}...
╚══════════════════════════════════════════════════════════════╝
"""
        return f"""
╔══════════════════════════════════════════════════════════════╗
║  {self.provider}: {self.model_name}
╠══════════════════════════════════════════════════════════════╣
║  ⏱️  Completion Time:     {self.completion_time:.2f}s
║  📝 Prompt Tokens:        {self.prompt_tokens:,}
║  ✍️  Completion Tokens:   {self.completion_tokens:,}
║  📊 Total Tokens:         {self.total_tokens:,}
║  ⚡ Speed:                {self.tokens_per_second:.1f} tokens/sec
╚══════════════════════════════════════════════════════════════╝
"""


In [18]:
class MultiModelClient:
    """Client for comparing completions across multiple LLM providers."""
    
    def __init__(self):
        # MiniMax
        try:
            self.minimax = MiniMaxClient()
            self.minimax_ready = True
        except Exception as e:
            self.minimax = None
            self.minimax_ready = False
            print(f"⚠️ MiniMax init failed: {e}")
        
        # OpenAI
        openai_key = os.getenv("OPENAI_API_KEY")
        self.openai = OpenAI(api_key=openai_key) if openai_key else None
        self.openai_ready = openai_key is not None
        
        # Anthropic
        anthropic_key = os.getenv("ANTHROPIC_API_KEY")
        self.anthropic = anthropic.Anthropic(api_key=anthropic_key) if anthropic_key else None
        self.anthropic_ready = anthropic_key is not None
        
        print("🔌 Initialized clients:")
        print(f"   {'✓' if self.minimax_ready else '✗'} MiniMax: {'Ready' if self.minimax_ready else 'No API key'}")
        print(f"   {'✓' if self.openai_ready else '✗'} OpenAI: {'Ready' if self.openai_ready else 'No API key'}")
        print(f"   {'✓' if self.anthropic_ready else '✗'} Anthropic: {'Ready' if self.anthropic_ready else 'No API key'}")
    
    def complete_minimax(
        self, 
        prompt: str, 
        system: str, 
        model: str = "MiniMax-M2.1",
        max_tokens: int = 8192
    ) -> CompletionResult:
        """Run completion with MiniMax."""
        if not self.minimax_ready:
            return CompletionResult(
                model_name=model, provider="MiniMax", content="",
                completion_time=0, prompt_tokens=0, completion_tokens=0,
                total_tokens=0, error="Client not initialized"
            )
        
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ]
        
        try:
            start = time.perf_counter()
            response = self.minimax.chat(messages, model=model, max_tokens=max_tokens, temperature=0.7)
            elapsed = time.perf_counter() - start
            
            return CompletionResult(
                model_name=model,
                provider="MiniMax",
                content=response.choices[0].message.content,
                completion_time=elapsed,
                prompt_tokens=response.usage.prompt_tokens,
                completion_tokens=response.usage.completion_tokens,
                total_tokens=response.usage.total_tokens
            )
        except Exception as e:
            return CompletionResult(
                model_name=model, provider="MiniMax", content="",
                completion_time=0, prompt_tokens=0, completion_tokens=0,
                total_tokens=0, error=str(e)
            )
    
    def complete_openai(
        self, 
        prompt: str, 
        system: str, 
        model: str = "gpt-4o", 
        max_tokens: int = 8192
    ) -> CompletionResult:
        """Run completion with OpenAI."""
        if not self.openai_ready:
            return CompletionResult(
                model_name=model, provider="OpenAI", content="",
                completion_time=0, prompt_tokens=0, completion_tokens=0,
                total_tokens=0, error="Client not initialized - set OPENAI_API_KEY"
            )
        
        # o1 models don't support system messages or temperature
        is_o1_model = model.startswith("o1")
        
        if is_o1_model:
            messages = [
                {"role": "user", "content": f"{system}\n\n{prompt}"}
            ]
        else:
            messages = [
                {"role": "system", "content": system},
                {"role": "user", "content": prompt}
            ]
        
        try:
            start = time.perf_counter()
            
            kwargs = {
                "model": model,
                "messages": messages,
                "max_completion_tokens": max_tokens,
            }
            
            if not is_o1_model:
                kwargs["temperature"] = 0.7
            
            response = self.openai.chat.completions.create(**kwargs)
            elapsed = time.perf_counter() - start
            
            return CompletionResult(
                model_name=model,
                provider="OpenAI",
                content=response.choices[0].message.content,
                completion_time=elapsed,
                prompt_tokens=response.usage.prompt_tokens,
                completion_tokens=response.usage.completion_tokens,
                total_tokens=response.usage.total_tokens
            )
        except Exception as e:
            return CompletionResult(
                model_name=model, provider="OpenAI", content="",
                completion_time=0, prompt_tokens=0, completion_tokens=0,
                total_tokens=0, error=str(e)
            )
    
    def complete_anthropic(
        self, 
        prompt: str, 
        system: str, 
        model: str = "claude-sonnet-4-20250514", 
        max_tokens: int = 8192
    ) -> CompletionResult:
        """Run completion with Anthropic."""
        if not self.anthropic_ready:
            return CompletionResult(
                model_name=model, provider="Anthropic", content="",
                completion_time=0, prompt_tokens=0, completion_tokens=0,
                total_tokens=0, error="Client not initialized - set ANTHROPIC_API_KEY"
            )
        
        try:
            start = time.perf_counter()
            response = self.anthropic.messages.create(
                model=model,
                max_tokens=max_tokens,
                system=system,
                messages=[{"role": "user", "content": prompt}]
            )
            elapsed = time.perf_counter() - start
            
            return CompletionResult(
                model_name=model,
                provider="Anthropic",
                content=response.content[0].text,
                completion_time=elapsed,
                prompt_tokens=response.usage.input_tokens,
                completion_tokens=response.usage.output_tokens,
                total_tokens=response.usage.input_tokens + response.usage.output_tokens
            )
        except Exception as e:
            return CompletionResult(
                model_name=model, provider="Anthropic", content="",
                completion_time=0, prompt_tokens=0, completion_tokens=0,
                total_tokens=0, error=str(e)
            )

# Initialize
client = MultiModelClient()


🔌 Initialized clients:
   ✓ MiniMax: Ready
   ✓ OpenAI: Ready
   ✓ Anthropic: Ready


## 🎯 Select Models to Compare

Configure which models you want to include in the comparison:


In [19]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎛️ CONFIGURATION: Select which models to compare
# ═══════════════════════════════════════════════════════════════════════════════

# Helper to pick models from the available list
def select_models(provider: str, model_patterns: list[str]) -> list[str]:
    """Select models matching patterns from available models."""
    available = AVAILABLE_MODELS.get(provider, [])
    selected = []
    for pattern in model_patterns:
        # Exact match first
        if pattern in available:
            selected.append(pattern)
        else:
            # Partial match (prefix)
            matches = [m for m in available if m.startswith(pattern)]
            selected.extend(matches[:1])  # Take first match
    return list(dict.fromkeys(selected))  # Remove duplicates, preserve order

# Option 1: Manual selection (uncomment models you want)
MODELS_TO_COMPARE = {
    # MiniMax models - uses all available by default
    "minimax": AVAILABLE_MODELS.get("minimax", [])[:1],  # First available
    
    # OpenAI models - pick specific ones
    "openai": select_models("openai", [
        "gpt-4o",           # Latest flagship
        # "gpt-4o-mini",    # Faster, cheaper
        # "gpt-4-turbo",    # Previous flagship  
        # "o1",             # Reasoning model
        # "o1-mini",        # Smaller reasoning model
    ]),
    
    # Anthropic models - pick specific ones
    "anthropic": select_models("anthropic", [
        "claude-sonnet-4-20250514",     # Latest Sonnet
        # "claude-3-5-sonnet-20241022", # Previous Sonnet
        # "claude-3-5-haiku",           # Fast and cheap
        # "claude-3-opus",              # Most capable
    ]),
}

# Option 2: Compare ALL available models (uncomment to use)
# MODELS_TO_COMPARE = AVAILABLE_MODELS.copy()

# Option 3: Quick comparison - one model per provider
# MODELS_TO_COMPARE = {
#     "minimax": AVAILABLE_MODELS.get("minimax", [])[:1],
#     "openai": select_models("openai", ["gpt-4o"]),
#     "anthropic": select_models("anthropic", ["claude-sonnet-4"]),
# }

print("📌 Models selected for comparison:")
total_models = 0
for provider, models in MODELS_TO_COMPARE.items():
    if models:
        print(f"\n  {provider.upper()}:")
        for model in models:
            print(f"    • {model}")
            total_models += 1
print(f"\n  Total: {total_models} models")

if total_models == 0:
    print("\n⚠️ No models selected! Check your API keys and MODELS_TO_COMPARE config.")


📌 Models selected for comparison:

  MINIMAX:
    • MiniMax-M2.1

  OPENAI:
    • gpt-4o

  ANTHROPIC:
    • claude-sonnet-4-20250514

  Total: 3 models


## 🌐 Website Generation Prompt

A moderately complex single-page website with:
- Modern responsive design
- Navigation, hero section, features grid
- Testimonials, pricing cards, contact form
- Animations and interactive elements
- Dark theme with accent colors


In [20]:
WEBSITE_SYSTEM_PROMPT = """You are an expert frontend developer. Generate complete, production-ready HTML with embedded CSS and JavaScript.

Requirements:
- Single HTML file with all styles and scripts embedded
- Modern, visually stunning design
- Fully responsive (mobile-first)
- Use CSS Grid and Flexbox
- Include smooth animations and micro-interactions
- Dark theme with vibrant accent colors
- Clean, semantic HTML5
- No external dependencies (no CDN links)

Output ONLY the HTML code, no explanations."""

WEBSITE_PROMPT = """Create a landing page for "NexusAI" - a fictional AI-powered productivity platform.

Include these sections:
1. **Navigation**: Sticky header with logo, nav links (Features, Pricing, About, Contact), and a glowing CTA button
2. **Hero Section**: Large headline, subheadline, email signup form, and an animated abstract background (CSS only)
3. **Features Grid**: 6 feature cards with icons (use emoji or CSS shapes), titles, and descriptions. Cards should have hover effects with subtle 3D transforms
4. **Stats Section**: 4 animated counters (Users: 50K+, Tasks Completed: 2M+, Time Saved: 10K hrs, Rating: 4.9★)
5. **Testimonials**: 3 testimonial cards with avatar placeholders, quotes, names, and roles. Include a subtle carousel effect
6. **Pricing**: 3 pricing tiers (Free, Pro $19/mo, Enterprise $49/mo) with feature lists and highlighted "popular" option
7. **Contact Form**: Name, email, message fields with validation styling and animated submit button
8. **Footer**: Logo, social links, copyright

Design requirements:
- Color scheme: Dark background (#0a0a0f), accent gradient (purple #8b5cf6 to cyan #06b6d4)
- Typography: System fonts with varied weights
- Glassmorphism effects on cards
- Animated gradient borders
- Smooth scroll behavior
- Parallax-like effects where appropriate
- Floating particles or geometric shapes in hero (CSS animations)
- All interactions should feel premium and polished"""

print("📝 Prompt ready!")
print(f"   System prompt: {len(WEBSITE_SYSTEM_PROMPT)} chars")
print(f"   User prompt: {len(WEBSITE_PROMPT)} chars")


📝 Prompt ready!
   System prompt: 481 chars
   User prompt: 1413 chars


In [21]:
# Run comparisons
results: dict[str, CompletionResult] = {}

print("🚀 Starting website generation comparison...")
print("=" * 70)

# MiniMax models
for model in MODELS_TO_COMPARE.get("minimax", []):
    print(f"\n⏳ Running MiniMax: {model}...")
    result = client.complete_minimax(WEBSITE_PROMPT, WEBSITE_SYSTEM_PROMPT, model=model)
    results[f"minimax_{model}"] = result
    print(result.summary())

# OpenAI models
for model in MODELS_TO_COMPARE.get("openai", []):
    print(f"\n⏳ Running OpenAI: {model}...")
    result = client.complete_openai(WEBSITE_PROMPT, WEBSITE_SYSTEM_PROMPT, model=model)
    results[f"openai_{model}"] = result
    print(result.summary())

# Anthropic models
for model in MODELS_TO_COMPARE.get("anthropic", []):
    print(f"\n⏳ Running Anthropic: {model}...")
    result = client.complete_anthropic(WEBSITE_PROMPT, WEBSITE_SYSTEM_PROMPT, model=model)
    results[f"anthropic_{model}"] = result
    print(result.summary())

print("\n" + "=" * 70)
print(f"✅ Completed {len(results)} model comparisons!")


🚀 Starting website generation comparison...

⏳ Running MiniMax: MiniMax-M2.1...

╔══════════════════════════════════════════════════════════════╗
║  MiniMax: MiniMax-M2.1
╠══════════════════════════════════════════════════════════════╣
║  ⏱️  Completion Time:     79.37s
║  📝 Prompt Tokens:        446
║  ✍️  Completion Tokens:   8,192
║  📊 Total Tokens:         8,638
║  ⚡ Speed:                103.2 tokens/sec
╚══════════════════════════════════════════════════════════════╝


⏳ Running OpenAI: gpt-4o...

╔══════════════════════════════════════════════════════════════╗
║  OpenAI: gpt-4o
╠══════════════════════════════════════════════════════════════╣
║  ⏱️  Completion Time:     28.50s
║  📝 Prompt Tokens:        447
║  ✍️  Completion Tokens:   3,463
║  📊 Total Tokens:         3,910
║  ⚡ Speed:                121.5 tokens/sec
╚══════════════════════════════════════════════════════════════╝


⏳ Running Anthropic: claude-sonnet-4-20250514...

╔════════════════════════════════════════════════

In [22]:
# Comparison summary table
try:
    import pandas as pd
    HAS_PANDAS = True
except ImportError:
    HAS_PANDAS = False

successful_results = {k: v for k, v in results.items() if v.success}

if successful_results:
    display(Markdown("## 📊 Comparison Summary"))
    
    if HAS_PANDAS:
        comparison_data = []
        for name, result in successful_results.items():
            comparison_data.append({
                'Provider': result.provider,
                'Model': result.model_name,
                'Time (s)': f"{result.completion_time:.2f}",
                'Completion Tokens': f"{result.completion_tokens:,}",
                'Speed (tok/s)': f"{result.tokens_per_second:.1f}",
                'Output Size': f"{len(result.content):,} chars"
            })
        
        df = pd.DataFrame(comparison_data)
        display(df)
    else:
        # Fallback without pandas
        print(f"{'Provider':<12} {'Model':<30} {'Time':<10} {'Tokens':<12} {'Speed':<12}")
        print("-" * 80)
        for name, result in successful_results.items():
            print(f"{result.provider:<12} {result.model_name:<30} {result.completion_time:.2f}s     {result.completion_tokens:<12,} {result.tokens_per_second:.1f} tok/s")
    
    # Find winners
    fastest = min(successful_results.values(), key=lambda x: x.completion_time)
    most_tokens = max(successful_results.values(), key=lambda x: x.completion_tokens)
    fastest_generation = max(successful_results.values(), key=lambda x: x.tokens_per_second)
    
    print(f"\n🏆 Results:")
    print(f"   ⚡ Fastest completion: {fastest.provider} {fastest.model_name} ({fastest.completion_time:.2f}s)")
    print(f"   🚀 Highest throughput: {fastest_generation.provider} {fastest_generation.model_name} ({fastest_generation.tokens_per_second:.1f} tok/s)")
    print(f"   📝 Most output: {most_tokens.provider} {most_tokens.model_name} ({most_tokens.completion_tokens:,} tokens)")
else:
    print("❌ No successful completions to compare.")


## 📊 Comparison Summary

,Provider,Model,Time (s),Completion Tokens,Speed (tok/s),Output Size
0,MiniMax,MiniMax-M2.1,79.37,"8,192",103.2,"33,920 chars"
1,OpenAI,gpt-4o,28.50,"3,463",121.5,"13,624 chars"



🏆 Results:
   ⚡ Fastest completion: OpenAI gpt-4o (28.50s)
   🚀 Highest throughput: OpenAI gpt-4o (121.5 tok/s)
   📝 Most output: MiniMax MiniMax-M2.1 (8,192 tokens)


In [23]:
def extract_html(content: str) -> str:
    """Extract HTML from model response, handling markdown code blocks and thinking tags."""
    
    # Remove <think>...</think> blocks (MiniMax specific)
    content = re.sub(r'<think>.*?</think>', '', content, flags=re.DOTALL)
    
    # Try to extract from code block
    patterns = [
        r'```html\s*([\s\S]*?)```',
        r'```\s*(<!DOCTYPE[\s\S]*?)```',
        r'```\s*(<html[\s\S]*?)```',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            return match.group(1).strip()
    
    # If no code block, check if content starts with HTML
    content = content.strip()
    if content.startswith('<!DOCTYPE') or content.startswith('<html'):
        return content
    
    return content

def display_website(result: CompletionResult, height: int = 600):
    """Display the generated website in an iframe."""
    if not result.success:
        print(f"❌ Cannot display - {result.provider} {result.model_name}: {result.error}")
        return
    
    html_content = extract_html(result.content)
    
    # Escape for iframe srcdoc
    escaped = html_content.replace('"', '&quot;')
    
    iframe = f'''
    <div style="margin: 20px 0;">
        <h3 style="color: #8b5cf6; margin-bottom: 10px; font-family: system-ui;">
            🌐 {result.provider} - {result.model_name}
            <span style="font-size: 0.7em; color: #666; margin-left: 10px;">
                ({result.completion_time:.2f}s • {result.completion_tokens:,} tokens)
            </span>
        </h3>
        <div style="border: 2px solid #333; border-radius: 8px; overflow: hidden;">
            <iframe srcdoc="{escaped}" 
                    style="width: 100%; height: {height}px; border: none; background: #0a0a0f;">
            </iframe>
        </div>
    </div>
    '''
    display(HTML(iframe))


## 🎨 Generated Websites

View the actual rendered output from each model:


In [24]:
# Display all generated websites
for name, result in results.items():
    if result.success:
        display_website(result, height=700)


In [14]:
# Code quality analysis
def analyze_html(content: str) -> dict:
    """Basic HTML quality metrics."""
    html = extract_html(content)
    
    return {
        'Total Lines': len(html.split('\n')),
        'Total Characters': len(html),
        'Has DOCTYPE': '<!DOCTYPE' in html.upper(),
        'Has Responsive Meta': 'viewport' in html.lower(),
        'Style Blocks': html.lower().count('<style'),
        'Script Blocks': html.lower().count('<script'),
        'Semantic Tags': sum(1 for tag in ['<header', '<nav', '<main', '<section', '<article', '<footer'] 
                           if tag in html.lower()),
        'Has Animations': 'animation' in html.lower() or '@keyframes' in html.lower(),
        'Uses Flexbox': 'flex' in html.lower(),
        'Uses Grid': 'display: grid' in html.lower() or 'display:grid' in html.lower(),
        'Has Gradients': 'gradient' in html.lower(),
        'Has Transitions': 'transition' in html.lower(),
    }

display(Markdown("## 🔍 Code Quality Analysis"))

for name, result in results.items():
    if result.success:
        print(f"\n{result.provider} - {result.model_name}")
        print("-" * 50)
        analysis = analyze_html(result.content)
        for metric, value in analysis.items():
            indicator = "✓" if value is True else ("✗" if value is False else value)
            print(f"  {metric}: {indicator}")


## 🔍 Code Quality Analysis


MiniMax - MiniMax-M2.1
--------------------------------------------------
  Total Lines: 1069
  Total Characters: 33148
  Has DOCTYPE: ✓
  Has Responsive Meta: ✓
  Style Blocks: 1
  Script Blocks: 1
  Semantic Tags: 3
  Has Animations: ✓
  Uses Flexbox: ✓
  Uses Grid: ✓
  Has Gradients: ✓
  Has Transitions: ✓

OpenAI - gpt-4o
--------------------------------------------------
  Total Lines: 422
  Total Characters: 11288
  Has DOCTYPE: ✓
  Has Responsive Meta: ✓
  Style Blocks: 1
  Script Blocks: 1
  Semantic Tags: 4
  Has Animations: ✓
  Uses Flexbox: ✓
  Uses Grid: ✓
  Has Gradients: ✓
  Has Transitions: ✓
